Analytics
===
*Bothound project*

# Initialization

In [65]:
# initialization
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn import preprocessing
from scipy.spatial.distance import cdist,pdist
from scipy.signal import argrelextrema
%matplotlib inline
from pylab import *
from numpy import *
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from scipy.stats import itemfreq

# boeh
from collections import OrderedDict
import pandas as pd
import bokeh.plotting as bk
bk.output_notebook()
from bokeh.charts import Bar
from bokeh.charts import Histogram

# enabling folding extension. Run it once.
import notebook
E = notebook.nbextensions.EnableNBExtensionApp()
E.enable_nbextension('usability/codefolding/main')

import yaml
from bothound_tools import BothoundTools

color_set = [
    [0, 0, 255],      #Blue
    [255, 0, 0],      #Red
    [0, 255, 0],      #Green
    [255, 255, 0],    #Yellow
    [255, 0, 255],    #Magenta
    [255, 128, 128],  #Pink
    [128, 128, 128],  #Gray
    [128, 0, 0],      #Brown
    [255, 128, 0],    #Orange
]

stram = open("../conf/bothound.yaml", "r")
conf = yaml.load(stram)
tools = BothoundTools(conf)
tools.connect_to_db()

def get_palette(N=5):
    result = []
    for x in range(N):
        s = color_set[x % len(color_set)]
        result.append([s[0]/255.0,s[1]/255.0,s[2]/255.0,1])
    return result
palette = get_palette(80)

def plot_costs(costs, num_clusters, title):
    KK = range(1,len(costs)+1)

    # elbow curve
    kIdx = num_clusters      
    clr = cm.spectral( np.linspace(0,1,10) ).tolist()
    mrk = 'os^p<dvh8>+x.'

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(KK, costs, 'b*-')
    ax.plot(num_clusters, costs[num_clusters-1], marker='o', markersize=14, 
        markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
    #ax.set_ylim((0,100))
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Average within sum of squeres')
    plt.title(title)
    
def plot_clusters(clusters, num_clusters,title="Histogram"):
    sizes = [0]*num_clusters
    for i in clusters: 
        if(i >= 0) :
            if (i >= num_clusters):
                print i
            sizes[i] = sizes[i]+1
    print (sizes)
    
      #plot histogramm
    left = [] 
    for i in range(len(sizes)):
        left.append(i-0.5)
    fig = plt.figure(figsize=(12,8))
    plt.title(title)
    ax = fig.add_subplot(111)
    ax.bar(left,sizes, color = palette)            
    
    
def get_clustering_model(X, num_clusters):
    model = KMeans(n_clusters=num_clusters, precompute_distances = True, max_iter = 500, n_init = 30)
    model.fit(X)
    
    clusters = model.predict(X)
    plot_clusters(clusters, num_clusters)
    return clusters

def get_best_clustering_model(X, max_number_of_clusters, title):
    cost = []
    KK = range(1,max_number_of_clusters+1)
    kms = []
    # calculate all the clustering and cost
    for no_of_clusters in KK:
        km = KMeans(n_clusters=no_of_clusters, precompute_distances = True, max_iter = 500, n_init = 30)
        km.fit(X)
        kms.append(km)

        sizes = [0]*no_of_clusters
        for i in km.predict(X): 
            if(i >= no_of_clusters):
                print i
            sizes[i] = sizes[i]+1
        print (sizes)

        cost.append(km.inertia_)

    # calculate first derivative
    derivative1 = [cost[i+1]-cost[i] for i in range(len(cost)-1)]
    #print "d1", derivative1

    # calculate second derivative
    derivative2 = [derivative1[i+1]-derivative1[i] for i in range(len(derivative1)-1)]
    #print "d2", derivative2

    max2 = argrelextrema(np.argsort(derivative2), np.less) 
    num_clusters = 4 
    #print "max2", max2
    if(len(max2[0]) > 0):
        num_clusters = max2[0][0] + 3
    else:
        # calculate third derivative
        derivative3 = [derivative2[i+1]-derivative2[i] for i in range(len(derivative2)-1)]
        #print derivative3

        max3 = argrelextrema(np.argsort(derivative3), np.greater) 
        if(len(max3[0]) > 0):
            num_clusters = max3[0][0] + 4 

    model = kms[num_clusters-1]
    
    # plot costs
    plot_costs(cost, model.n_clusters, "Cost of k-Means." + title)

    clusters = model.predict(X)
    plot_clusters(clusters, model.n_clusters, title)
    return clusters, model.n_clusters, cost


import plotly
plotly.offline.init_notebook_mode() # run at the start of every notebook

from plotly.plotly import iplot
from plotly.graph_objs import Scatter3d, Data, Marker
import plotly.graph_objs as go

def plot3(feature_indexes, X, clusters, selected_cluster, title = "Cluster"):
    clusters_plot = []
    num_clusters = max(clusters)+1
    for i in range(0, num_clusters):
        d = X[clusters == i,: ]
        cluster = Scatter3d(
            x=d[:,feature_indexes[0]],
            y=d[:,feature_indexes[1]],
            z=d[:,feature_indexes[2]],
            mode='markers',
            name = "All traffic" if i == 0 else "{} {}".format(title, i),
            marker=dict(
                color='rgb({}, {}, {})'.format(palette[i][0]*255,palette[i][1]*255,palette[i][2]*255 ),
                size=12,
                line=dict(
                    color='rgb(204, 204, 204)',
                    width=0.0
                ),
                opacity=0.2
            )
        )
        clusters_plot.append(cluster)

    data = Data(clusters_plot)
    bk_color = "rgb(224, 224, 224)"
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0,t=60),
        title='', 
        height = 1000,
        width = 1000,
        legend=dict(
            #x=0,
            #y=1,
            #traceorder='normal',
            font=dict(
                family='sans-serif',
                size=16,
                color='#000'
            ),
            bgcolor='#E2E2E2',
            bordercolor='#FFFFFF',
            borderwidth=2
        ),
        scene=go.Scene(
            xaxis=dict(
                title = features[feature_indexes[0]],
    showbackground=True, # (!) show axis background
    backgroundcolor=bk_color, # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
           ),
            yaxis=dict(
                 title = features[feature_indexes[1]],
    showbackground=True, # (!) show axis background
    backgroundcolor=bk_color, # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
            ),
            zaxis=dict(
                 title = features[feature_indexes[2]],
    showbackground=True, # (!) show axis background
    backgroundcolor=bk_color, # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
            )
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)


def plot_intersection(clusters, num_clusters, id_incident, ips, id_incident2, cluster2 = -1):
    clusters_np = np.array(clusters)
    ips_np = np.array(ips)
    ips2 = set(tools.get_ips(id_incident2, cluster2))
    d = {}
    d["Cluster"] = []
    d["Incident"] = []
    d["data"] = []
    percentages = []
    intersections = []
    
    for cluster in range(0, num_clusters):
        d["Cluster"].append(cluster)
        d["Incident"].append("Unique from incident {}".format(id_incident))
        cluster_ips = set(ips_np[clusters_np == cluster])
        intersection = len(ips2.intersection(cluster_ips))
        intersections.append(intersection)
        d["data"].append(len(cluster_ips)-intersection)
        if(len(cluster_ips) == 0):
            percentages.append(0)
        else:
            percentages.append(intersection*100.0/len(cluster_ips))
        
    for cluster in range(0, num_clusters):
        d["Cluster"].append(cluster)
        d["Incident"].append("Intersection with incident {}".format(id_incident2))
        d["data"].append(intersections[cluster])
        
    df=pd.DataFrame(d)
    p=Bar(df,label='Cluster',values='data',stack='Incident',legend='top_right', 
          title = "Intersection. Incident {} vs. Incident {} (cluster={})".format(id_incident, id_incident2, cluster2) ,
         ylabel = "IP sessions", plot_width=1000, plot_height=600)
    bk.show(p)   

def plot_countries(clusters, num_clusters, sessions, num_countries = 10):
    countries = tools.get_countries()
    ids = np.array([s['id_country'] for s in sessions])
    #first find the best countries 
    if(num_countries > len(countries)):
        num_countries = len(countries)
      
    # find the most ccountries count in total
    freq = itemfreq(ids)
    sorted_countries = sorted(freq, key=lambda k: k[1], reverse=True) 
    best_countries = sorted_countries[0:num_countries]
    codes = []
    for i in range(0,len(best_countries)):
        c = (item for item in countries if item["id"] == best_countries[i][0]).next()
        codes.append(c)

    # calculate best countries count per cluster
    clusters_np = np.array(clusters)
    d = {}
    d["Cluster"] = []
    d["Country"] = []
    d["data"] = []
    freqs= []
    for cluster in range(0, num_clusters):
        ids_cluster = ids[clusters_np == cluster]
        freqs.append(itemfreq(ids_cluster))
        
    for i in range(0,len(best_countries)):
        for cluster in range(0, num_clusters):
            d["Cluster"].append(cluster)
            d["Country"].append(codes[i]["name"])
            exists = False
            for f in freqs[cluster]:
                if (f[0] == best_countries[i][0]):
                    d["data"].append(f[1])
                    exists = True
                    break
            if (not exists) :
                d["data"].append(0)

    df=pd.DataFrame(d)
    p=Bar(df,label='Cluster',values='data',stack='Country',legend='top_right', 
          title = "Countries" ,
         ylabel = "IP sessions", plot_width=1000, plot_height=600)
    bk.show(p)   
    
def plot_ban(clusters, num_clusters, sessions):
    clusters_np = np.array(clusters)
    bans = np.array([s['ban'] for s in sessions])
    d = {}
    d["Cluster"] = []
    d["Ban"] = []
    d["data"] = []
    banned = []
    percentage = []
    
    for cluster in range(0, num_clusters):
        d["Cluster"].append(cluster)
        d["Ban"].append("Served")
        cluster_total = bans[clusters_np == cluster]
        cluster_banned = cluster_total[cluster_total==1]
        banned.append(cluster_banned.shape[0])
        if (cluster_total.shape[0] == 0):
            p = 0
        else:
            p = float("{0:.2f}".format(cluster_banned.shape[0]*100.0/cluster_total.shape[0]))
        percentage.append(p)
        d["data"].append(cluster_total.shape[0]-cluster_banned.shape[0])
          
    for cluster in range(0, num_clusters):
        d["Cluster"].append(cluster)
        d["Ban"].append("Banned")
        d["data"].append(banned[cluster])

    df=pd.DataFrame(d)
    p=Bar(df,label='Cluster',values='data',stack='Ban',legend='top_right', 
          title = "Banjax Ban" ,
         ylabel = "IP sessions", plot_width=1000, plot_height=600)
    bk.show(p)   
    print banned
    print percentage


Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py:78: DeprecationWarning:

JupyterApp._config_dir_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py:340: DeprecationWarning:

EnableNBExtensionApp._config_file_name_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py:91: DeprecationWarning:

JupyterApp._data_dir_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py:73: DeprecationWarning:

JupyterApp._jupyter_path_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py:69: DeprecationWarning:

JupyterApp._log_level_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py:97: DeprecationWarning:

JupyterApp._runtime_dir_default is depre

# Configuration

In [66]:
#id_incidents = [27,19] # Kotsubynske
#id_incidents = [24,25,26,19,27] # Kotsubynske

#id_incidents = [29,30,31,32,33,34]
id_incidents = [29,30]

# Read Data

In [67]:
# Reading from Database
incidents = []
for id in id_incidents:
    print "Indicent", id, "loading..."
    incident = {}
    incident["id"] = id
    incident["sessions"] = tools.get_sessions(id)
    incident["incident"] = tools.get_incident(id)[0]
    incidents.append(incident)
    print "total sessions", len(incident['sessions'])
print "Done."

Indicent 29 loading...
total sessions 22914
Indicent 30 loading...
total sessions 12352
Done.


# Banjax intersection

In [56]:
# banjax
def get_unique_sessions(sessions):
    ips = {}
    for s in sessions:
        ips[s["IP"]] = s
    res = []
    for ip in ips.keys():
        res.append(ips[ip])
    return res
        

# calculate the common set
common = -1 
common_ban = -1 

print "incident id,total IPs,common IPs,banned IPs,common banned IPs,start,stop"
for incident in incidents:
    sessions = incident['sessions']
    
    #common
    ips = set(np.array([s["IP"] for s in sessions]))
    #print len(ips)
    if(common < 0):
       common = ips
    else:
       common = common.intersection(ips)

    #common ban
    banned = list(filter(lambda x: x['ban'] == 1, sessions))
    ips_ban = set(np.array([s["IP"] for s in banned]))
    if(common_ban < 0):
       common_ban = ips_ban
    else:
       common_ban = common_ban.intersection(ips_ban)
    #print incident['incident'][0]
    print "{},{},{},{},{},{},{}".format(incident["id"], len(ips), len(common), len(ips_ban), len(common_ban),
                                 incident['incident']['start'], incident['incident']['stop'])
    
print "---"

print "id,% common,% common ban"
for incident in incidents:
    sessions = incident['sessions']

    ips = set(np.array([s["IP"] for s in sessions]))
    
    banned = list(filter(lambda d: d['ban'] == 1, sessions))
    ips_ban = set(np.array([s["IP"] for s in banned]))

    intersect = ips.intersection(common) 
    intersect_ban = ips_ban.intersection(common_ban) 
    
    #print incident["id"], len(intersect) * 100.0 / len(ips), '%', len(intersect_ban) * 100.0 / len(ips_ban), '%'
    print "{},{:.2f}%,{:.2f}%".format(incident["id"], len(intersect) * 100.0 / len(ips), 
                                    len(intersect_ban) * 100.0 / len(ips_ban))

    

incident id,total IPs,common IPs,banned IPs,common banned IPs,start,stop
31,5948,5948,2286,2286,2016-03-01 15:00:00,2016-03-01 19:30:00
32,3887,3132,2279,1719,2016-03-02 12:30:00,2016-03-02 16:00:00
33,9670,817,5251,219,2016-03-04 09:00:00,2016-03-04 09:30:00
34,7878,459,3074,83,2016-03-08 14:20:00,2016-03-08 16:40:00
---
id,% common,% common ban
31,7.72%,3.63%
32,11.81%,3.64%
33,4.75%,1.58%
34,5.83%,2.70%


# User Agent

In [71]:
def plot_countries(incidents, num_countries = 25):
    countries = tools.get_countries()
    ids = []
    for incident in incidents:
        for s in incident['sessions']:
            if(s['ban'] == 1) :
                ids.append(s['id_country'])
    ids = np.array(ids)
    #first find the best countries 
    if(num_countries > len(countries)):
        num_countries = len(countries)
      
    # find the most ccountries count in total
    freq = itemfreq(ids)
    sorted_countries = sorted(freq, key=lambda k: k[1], reverse=True) 
    best_countries = sorted_countries[0:num_countries]
    codes = []
    for i in range(0,len(best_countries)):
        c = (item for item in countries if item["id"] == best_countries[i][0]).next()
        codes.append({ "count": best_countries[i][1], "name": c['name'], "id": c["id"]})
        #codes.append([best_countries[i][1], c['name']])
    for c in codes:
        print c


In [72]:
plot_countries(incidents)

{'count': 9264, 'name': 'Russian Federation', 'id': 11L}
{'count': 4026, 'name': 'Ukraine', 'id': 6L}
{'count': 2684, 'name': 'China', 'id': 53L}
{'count': 1716, 'name': 'United States', 'id': 2L}
{'count': 1282, 'name': 'Germany', 'id': 8L}
{'count': 1209, 'name': 'Lithuania', 'id': 75L}
{'count': 951, 'name': 'United Kingdom', 'id': 15L}
{'count': 752, 'name': 'Switzerland', 'id': 40L}
{'count': 703, 'name': 'Gibraltar', 'id': 145L}
{'count': 693, 'name': 'France', 'id': 20L}
{'count': 661, 'name': None, 'id': 17L}
{'count': 565, 'name': 'Netherlands', 'id': 5L}
{'count': 482, 'name': 'Turkey', 'id': 13L}
{'count': 387, 'name': 'Belarus', 'id': 49L}
{'count': 383, 'name': 'Japan', 'id': 36L}
{'count': 365, 'name': 'Indonesia', 'id': 21L}
{'count': 358, 'name': None, 'id': 58L}
{'count': 333, 'name': 'United Arab Emirates', 'id': 14L}
{'count': 324, 'name': 'Poland', 'id': 18L}
{'count': 319, 'name': 'Croatia', 'id': 27L}
{'count': 297, 'name': 'Korea', 'id': 37L}
{'count': 243, 'name

In [63]:
def plot_ua(incidents):
    ua = "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)"
    id_countries = [1,10,35,21,7,26,34,13,32,37]
    countries = tools.get_countries()
    names = " "
    for i in id_countries:
        c = (item for item in countries if item["id"] == i).next()
        names = names + c['name'] + ","
    
    d = {}
    d["Incident"] = []
    d["Marker"] = []
    d["data"] = []
    attackers = []
    percentage = []
    
    i = 1
    total = 0
    bad = 0
    for incident in incidents:
        #print incident['incident']['id']
        d["Incident"].append(i)
        i = i + 1
        d["Marker"].append("Other")

        sessions = incident['sessions']
        num_countries = 0
        num_bad = 0
        num_total = 0
        for s in sessions:
            #if(s['ban'] == 1) :
            if(True) :
                num_total = num_total + 1
                #if(s['id_country'] in id_countries):
                #    num_countries = num_countries + 1
                #if(s['ua'] == ua):
                print s['ua']
                if (s['ua'].find("WordPress") >= 0 if s['ua'] is not None else False):
                    num_bad = num_bad + 1
                    
        d["data"].append(num_total - num_bad)
        attackers.append(num_bad)
        total = total + num_total
        bad = bad + num_bad
        percentage.append(100.0*num_bad/num_total)
    i = 1
    for incident, attacker in zip(incidents, attackers):
        d["Incident"].append(i)
        i = i + 1
        d["Marker"].append(names)
        d["data"].append(attacker)

    df=pd.DataFrame(d)
    p=Bar(df,label='Incident',values='data',stack='Marker',legend='top_right', 
        title = "Banned IPs by countries, {:.1f}%".format(bad*100.0/total),
        ylabel = "Unique IPs", plot_width=1000, plot_height=1000)
    bk.show(p)   
    #print banned
    print percentage
    print names

In [64]:
plot_ua(incidents)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


/usr/local/lib/python2.7/dist-packages/bokeh/core/properties.py:453: DeprecationWarning:

Setting a fixed font size value as a string '10pt' is deprecated, set with value('10pt') or ['10pt'] instead

/usr/local/lib/python2.7/dist-packages/bokeh/core/properties.py:453: DeprecationWarning:

Setting a fixed font size value as a string '14pt' is deprecated, set with value('14pt') or ['14pt'] instead



[0.0, 0.0, 0.0, 0.0]
 Vietnam,India,Iran,Indonesia,Saudi Arabia,Philippines,Mexico,Turkey,Thailand,Korea,


/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:52: DeprecationWarning:

Comm._comm_id_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:29: DeprecationWarning:

Comm._iopub_socket_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:24: DeprecationWarning:

Comm._kernel_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:32: DeprecationWarning:

Comm._session_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:41: DeprecationWarning:

Comm._topic_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:24: DeprecationWarning:

Comm._kernel_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/com

# Hit rate

In [41]:
hit_rate = []
hit_rate_ua = []
ua = "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)"
for incident in incidents:
    for s in incident['sessions']:
        v = s['request_interval']
        if(v == 1800):
            v = 0
        if(v != 0):
            v = 60.0/v
        #if(s['ua'] == ua) :
        if(s['ban'] == 1) :
            hit_rate_ua.append(v)
        else:
            hit_rate.append(v)
            
trace_other = go.Box(
    y = hit_rate,
    boxpoints='all',
    jitter=0.5,
    name='Others',
    pointpos=-1.8
)

trace_ua = {}
trace_ua = go.Box(
    y = hit_rate_ua,
    boxpoints='all',
    jitter=0.5,
    name='banned IPs',
    pointpos=-1.8
)

data = Data([trace_other, trace_ua])
layout = go.Layout(
    showlegend=False,
    height = 900,
    title='Hit rate of banned IPs',
    xaxis=go.XAxis(
        showgrid=True,
        showline=True,
        ticks=''
    ),
    yaxis=go.YAxis(
        showline=True,
        ticks='',
        zeroline=True,
        range = [0,300],
        title = "Hit rate/minute"
    )
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)


# Cross incident plot

In [50]:
features = [
    "request_interval", #1
    #"ua_change_rate",#2
    "html2image_ratio",#3
    "variance_request_interval",#4
    #"payload_average",#5
    "error_rate",#6
    #"request_depth",#7
    "request_depth_std",#8
    "session_length",#9
    #"percentage_cons_requests",#10
]

values = []
incident_indexes = []
i = 1
for incident in incidents:
    for s in incident['sessions']:
        
        row = []
        for f in features:
            row.append(s[f])
            
        row.append(s['ban'])
        if(s['ban'] == 0):
            incident_indexes.append(0) 
        else:
            incident_indexes.append(i) 
        
        values.append(row)
    i = i + 1
    
X = np.array(values)
incident_indexes = np.array(incident_indexes)
X.shape

(3387, 7)

In [51]:
plot3([3,1,2], X, incident_indexes, -1, "Banned from Incident")

# Response histogram

In [25]:
def plot_histogram(data, x_label, y_label, title):
   
    keys = []
    for d in data:
        keys = keys + d["values"].keys()
    keys = set(keys)

    d = {}
    d[x_label] = []
    d["legend"] = []
    d["data"] = []
    
    for x in data:    
        for key in keys:
            d[x_label].append(x["x"])
            d["legend"].append(key)
            d["data"].append(x["values"][key] if key in x["values"] else 0)
    df=pd.DataFrame(d)
    p=Bar(df,label='Incident',values='data',stack="legend",legend='top_right', 
        title = title,
        ylabel = y_label, plot_width=1000, plot_height=1000)
    bk.show(p)   


In [86]:
#kotsubynske
data = []
data.append({"x" : 1, "values" : {"200" : 121638, "404" : 31436, "503" : 7112}})
data.append({"x" : 2, "values" : {"200" : 35853 , "403" : 138}})
data.append({"x" : 3, "values" : {"200" : 23472  , "403" : 531, "404" : 25168 }})
data.append({"x" : 4, "values" : {"200" : 16677   , "503" : 22792 , "404" : 1495610  }})
data.append({"x" : 5, "values" : {"200" : 347919    , "403" : 1570  , "404" : 42429   }})

plot_histogram(data, "Incident", "Hits", "Responce code distribution")

/usr/local/lib/python2.7/dist-packages/bokeh/core/properties.py:453: DeprecationWarning:

Setting a fixed font size value as a string '10pt' is deprecated, set with value('10pt') or ['10pt'] instead

/usr/local/lib/python2.7/dist-packages/bokeh/core/properties.py:453: DeprecationWarning:

Setting a fixed font size value as a string '14pt' is deprecated, set with value('14pt') or ['14pt'] instead



/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:52: DeprecationWarning:

Comm._comm_id_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:29: DeprecationWarning:

Comm._iopub_socket_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:24: DeprecationWarning:

Comm._kernel_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:32: DeprecationWarning:

Comm._session_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:41: DeprecationWarning:

Comm._topic_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:24: DeprecationWarning:

Comm._kernel_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/com

# Spider devices in bdsmovement

In [28]:
data = []
data.append({"x" : 1, "values" : {"Other" : 712, " Spider" : 12}})
data.append({"x" : 2, "values" : {"Other" : 673, " Spider" : 11}})
data.append({"x" : 3, "values" : {"Other" : 2289, " Spider" : 2229}})
data.append({"x" : 4, "values" : {"Other" : 12974, " Spider" : 12869}})
data.append({"x" : 5, "values" : {"Other" : 5251, " Spider" : 5014}})
data.append({"x" : 6, "values" : {"Other" : 3074, " Spider" : 3019}})
for d in data:
    d["values"]["Other"] = d["values"]["Other"] - d["values"][" Spider"]

print 23131 * 100.0 / (2289+12974+5251+3074)

plot_histogram(data, "Incident", "Number of IPs", "Portion of Spider devices")

98.0625741903


/usr/local/lib/python2.7/dist-packages/bokeh/core/properties.py:453: DeprecationWarning:

Setting a fixed font size value as a string '10pt' is deprecated, set with value('10pt') or ['10pt'] instead

/usr/local/lib/python2.7/dist-packages/bokeh/core/properties.py:453: DeprecationWarning:

Setting a fixed font size value as a string '14pt' is deprecated, set with value('14pt') or ['14pt'] instead



/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:52: DeprecationWarning:

Comm._comm_id_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:29: DeprecationWarning:

Comm._iopub_socket_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:24: DeprecationWarning:

Comm._kernel_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:32: DeprecationWarning:

Comm._session_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:41: DeprecationWarning:

Comm._topic_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/comm.py:24: DeprecationWarning:

Comm._kernel_default is deprecated: use @default decorator instead.

/usr/local/lib/python2.7/dist-packages/ipykernel/comm/com

# Intersection BTSELEM vs BDSMOVEMENT

In [ ]:
data = []
data.append({"x" : 1, "values" : {"Bdsmovement.org" : 713, " Identical to Btselem.org" : 12}})
data.append({"x" : 2, "values" : {"Bdsmovement.org" : 673, " Identical to Btselem.org" : 11}})
data.append({"x" : 3, "values" : {"Bdsmovement.org" : 2289, " Identical to Btselem.org" : 2229}})
data.append({"x" : 4, "values" : {"Bdsmovement.org" : 12974, " Identical to Btselem.org" : 12869}})
data.append({"x" : 5, "values" : {"Bdsmovement.org" : 5251, " Identical to Btselem.org" : 5014}})
data.append({"x" : 6, "values" : {"Bdsmovement.org" : 3074, " Identical to Btselem.orgr" : 3019}})
for d in data:
    d["values"]["Bdsmovement.org"] = d["values"]["Bdsmovement.org"] - d["values"][" Identical to Btselem.org"]


plot_histogram(data, "Incident", "Number of IPs", "Bdsmovement.org incidents with highlited identical IPs with Btselem.org incident")